In [9]:
import requests
import os
import pandas as pd
from datetime import datetime
import pytz

oc = os.getenv('OC', '')

In [2]:
df = pd.read_csv("data/법령검색목록_법령_건축.csv", skiprows=1)
print(len(df))
d건축법 시행령건축법 시행령건축법 시from architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_lafrom architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_lawsfrom architecture_agent.ingestion.fetch_law import fetch_and_save_law

42


,순번,법령MST,소관부처코드,소관부처명,법령ID,법령명,공포일자,공포번호,시행일자,법령구분코드,법령구분명,법령분야코드,법령분야명
0,1,276923,1613000,국토교통부,10594,건축기본법,20251001,제21065호,20251001,A0002,법률,34030000,第3章 建 築
1,2,277915,1613000,국토교통부,10787,건축기본법 시행령,20251001,제35811호,20251001,A0007,대통령령,34030000,第3章 建 築
2,3,266691,1613000,국토교통부,13478,건축물관리법,20241203,제20549호,20250604,A0002,법률,34030000,第3章 建 築
3,4,271337,1613000,국토교통부,13742,건축물관리법 시행령,20250527,제35549호,20250604,A0007,대통령령,34030000,第3章 建 築
4,5,271531,1613000,국토교통부,13747,건축물관리법 시행규칙,20250602,제1495호,20250602,A0103,국토교통부령,34030000,第3章 建 築


In [10]:
id = "001823"
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID=001823&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()

In [29]:
# 조문 파싱
import re
from dataclasses import dataclass, field


@dataclass
class ArticleChunk:
    """조 단위 chunk"""
    # 식별
    law_name: str                    # "건축법"
    law_id: str                      # "001823"
    article_num: str                 # "46"
    article_title: str               # "건축선의 지정"
    
    # 본문
    content: str                     # 항+호+목 플랫 결합 텍스트
    content_resolved: str = ""       # 축약어 치환 버전 (STEP 1-5에서 채움)
    
    # 계층 구조 보존 (metadata용)
    paragraphs: list = field(default_factory=list)
    # [
    #   {"num": "1", "content": "...", 
    #    "subs": [{"num": "1", "content": "...", 
    #              "items": [{"num": "가", "content": "..."}]}]}
    # ]
    
    # 참조 (STEP 1-4에서 채움)
    internal_refs: list = field(default_factory=list)
    external_refs: list = field(default_factory=list)
    parent_law_refs: list = field(default_factory=list)
    
    # 축약어맵 (STEP 1-3에서 채움)
    abbreviations: dict = field(default_factory=dict)
    
    # 메타
    effective_date: str = ""
    change_type: str = ""


def parse_law_data(data: dict) -> list[ArticleChunk]:
    """
    법령 API 응답 전체를 파싱하여 ArticleChunk 리스트 반환
    """
    law_info = data["법령"]["기본정보"]
    law_name = law_info["법령명_한글"]
    law_id = law_info["법령ID"]
    
    articles_raw = data["법령"]["조문"]["조문단위"]
    
    chunks = []
    for article in articles_raw:
        chunk = parse_article(article, law_name, law_id)
        if chunk:
            chunks.append(chunk)
    
    return chunks


def parse_article(article: dict, law_name: str, law_id: str) -> ArticleChunk | None:
    """단일 조문을 ArticleChunk로 변환"""
    
    # 조문 여부 확인 (부칙 등 제외)
    if article.get("조문여부") != "조문":
        return None
    
    article_num = article.get("조문번호", "")
    article_title_raw = article.get("조문제목", "")
    article_header = article.get("조문내용", "")  # "제2조(정의)"
    
    # 항 파싱
    paragraphs_structured = []
    content_parts = [article_header]  # 플랫 텍스트 시작
    
    paragraphs_raw = normalize_to_list(article.get('항'))

    for para in paragraphs_raw:
        para_num = normalize_paragraph_num(para.get("항번호", ""))
        para_content = para.get("항내용", "")
        
        content_parts.append(para_content)
        
        subs_structured = []
        
        # 호 파싱
        for sub in para.get("호", []):
            sub_num = sub.get("호번호", "").strip().rstrip(".")
            sub_content = sub.get("호내용", "")
            
            content_parts.append(sub_content)
            
            items_structured = []
            
            # 목 파싱
            for item in sub.get("목", []):
                item_num = item.get("목번호", "").strip().rstrip(".")
                item_content = item.get("목내용", "")
                
                content_parts.append(item_content)
                items_structured.append({
                    "num": item_num,
                    "content": item_content
                })
            
            subs_structured.append({
                "num": sub_num,
                "content": sub_content,
                "items": items_structured
            })
        
        paragraphs_structured.append({
            "num": para_num,
            "content": para_content,
            "subs": subs_structured
        })
    
    # 플랫 텍스트 결합
    content = "\n".join(content_parts)
    
    return ArticleChunk(
        law_name=law_name,
        law_id=law_id,
        article_num=article_num,
        article_title=article_title_raw,
        content=content,
        paragraphs=paragraphs_structured,
        effective_date=article.get("조문시행일자", ""),
        change_type=article.get("조문제개정유형", ""),
    )

def normalize_to_list(value):
    """항/호/목이 dict로 올 수도, list로 올 수도, 없을 수도 있는 경우 처리"""
    if value is None:
        return []
    if isinstance(value, dict):
        return [value]
    if isinstance(value, list):
        return value
    return []


def normalize_paragraph_num(raw: str) -> str:
    """①②③... → 1, 2, 3..."""
    circled_map = {
        "①": "1", "②": "2", "③": "3", "④": "4", "⑤": "5",
        "⑥": "6", "⑦": "7", "⑧": "8", "⑨": "9", "⑩": "10",
        "⑪": "11", "⑫": "12", "⑬": "13", "⑭": "14", "⑮": "15",
    }
    raw = raw.strip()
    return circled_map.get(raw, raw)

In [27]:
# 축약어 맵
def extract_abbreviations(chunks: list[ArticleChunk]) -> dict:
    """
    전체 조문에서 축약어 맵 추출
    "이하 "건축선"이라 한다" 같은 패턴
    
    Returns:
        {"건축선": "건축물을 건축할 수 있는 선", "시·도지사": "특별시장·광역시장...", ...}
    """
    
    # 전체 텍스트 합치기 (법률 전체를 한 번 스캔)
    full_text = "\n".join(chunk.content for chunk in chunks)
    
    abbr_map = {}
    
    patterns = [
        # [이하 "건축선(建築線)"이라 한다]
        r'([^[」\n]+?)\[이하\s*["\u201C]([^"\u201D]+)["\u201D]\s*(?:이라|라)\s*한다\]',
        # (이하 "시·도지사"라 한다)
        r'([^(」\n]+?)\(이하\s*["\u201C]([^"\u201D]+)["\u201D]\s*(?:이라|라)\s*한다\)',
        # 이하 "건축물의 건축등"이라 한다
        r'([^,\n]+?)\s*이하\s*["\u201C]([^"\u201D]+)["\u201D]\s*(?:이라|라)\s*한다',
    ]
    
    for pattern in patterns:
        for match in re.finditer(pattern, full_text):
            full_name = match.group(1).strip()
            short_name = match.group(2).strip()
            
            # 너무 긴 full_name은 정리 (앞부분 잘라냄)
            # "도로와 접한 부분에 건축물을 건축할 수 있는 선" → 마지막 의미 단위만
            if len(full_name) > 50:
                full_name = full_name[-50:]
            
            abbr_map[short_name] = full_name
    
    # 모든 chunk에 축약어맵 첨부
    for chunk in chunks:
        chunk.abbreviations = abbr_map
    
    return abbr_map

In [15]:
# 참조 추출

In [28]:
def extract_references(chunks: list[ArticleChunk], law_name: str):
    """
    각 조문에서 참조 관계를 추출하여 chunk에 채움
    
    3종류:
    - internal_refs: 같은 법 내 참조 ("제2조제1항제11호")
    - external_refs: 다른 법 참조 ("「국토의 계획 및 이용에 관한 법률」 제76조")
    - parent_law_refs: 시행령→본법 참조 ("법 제46조제1항에 따라")
    """
    
    # 다른 법률 참조: 「법률명」 + 선택적으로 제X조
    external_pattern = r'「([^」]+)」(?:\s*제(\d+(?:의\d+)?)조)?(?:\s*제(\d+)항)?'
    
    # 같은 법 내부 참조: 제X조(제Y항)(제Z호)
    internal_pattern = r'제(\d+(?:의\d+)?)조(?:제(\d+)항)?(?:제(\d+)호)?'
    
    # 시행령에서 본법 참조: "법 제X조"
    parent_pattern = r'(?<![가-힣])법\s+제(\d+(?:의\d+)?)조(?:제(\d+)항)?'
    
    for chunk in chunks:
        text = chunk.content
        
        # ── external_refs ──
        for m in re.finditer(external_pattern, text):
            ref_law = m.group(1)
            # 자기 자신 법률 참조는 제외
            if ref_law == law_name:
                continue
            chunk.external_refs.append({
                "law_name": ref_law,
                "article": m.group(2),
                "paragraph": m.group(3),
                "raw": m.group(0),
            })
        
        # ── internal_refs ──
        # 외부 참조로 이미 잡힌 부분 제거한 텍스트에서 추출
        text_no_external = re.sub(external_pattern, "", text)
        # parent_ref 패턴도 제거 (시행령인 경우)
        text_clean = re.sub(parent_pattern, "", text_no_external)
        
        for m in re.finditer(internal_pattern, text_clean):
            ref_article = m.group(1)
            # 자기 자신 조문 참조는 제외
            if ref_article == chunk.article_num:
                continue
            chunk.internal_refs.append({
                "article": ref_article,
                "paragraph": m.group(2),
                "subparagraph": m.group(3),
                "raw": m.group(0),
            })
        
        # ── parent_law_refs (시행령 전용) ──
        for m in re.finditer(parent_pattern, text):
            chunk.parent_law_refs.append({
                "article": m.group(1),
                "paragraph": m.group(2),
                "raw": m.group(0),
            })
        
        # 중복 제거
        chunk.internal_refs = deduplicate_refs(chunk.internal_refs)
        chunk.external_refs = deduplicate_refs(chunk.external_refs)
        chunk.parent_law_refs = deduplicate_refs(chunk.parent_law_refs)


def deduplicate_refs(refs: list[dict]) -> list[dict]:
    """raw 기준으로 중복 제거"""
    seen = set()
    result = []
    for ref in refs:
        key = ref.get("raw", str(ref))
        if key not in seen:
            seen.add(key)
            result.append(ref)
    return result

In [17]:
# 축약어 치환

In [18]:
def resolve_abbreviations(chunks: list[ArticleChunk], abbr_map: dict):
    """축약어를 원래 명칭으로 치환한 content_resolved 생성"""
    
    # 긴 축약어부터 치환 (부분 매칭 방지)
    sorted_abbrs = sorted(abbr_map.items(), key=lambda x: len(x[0]), reverse=True)
    
    for chunk in chunks:
        resolved = chunk.content
        
        for short, full in sorted_abbrs:
            if len(short) <= 2:
                # "법", "영" 같은 짧은 축약어: 단어 경계 체크
                pattern = rf'(?<![가-힣a-zA-Z]){re.escape(short)}(?=\s|제|의|에|을|를|이|가|은|는|과|와|으로|$)'
            else:
                pattern = re.escape(short)
            
            resolved = re.sub(pattern, full, resolved)
        
        chunk.content_resolved = resolved

In [19]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_naver import ChatClovaX, ClovaXEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


def store_to_qdrant(chunks: list[ArticleChunk], 
                     collection_name: str = "building_law"):
    """ArticleChunk 리스트를 Qdrant에 저장"""
    
    # Qdrant 클라이언트
    client = QdrantClient(path="./qdrant_data")  # 로컬 저장
    
    # 컬렉션 생성 (없으면)
    if not client.collection_exists(collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(
                size=1024, 
                distance=Distance.COSINE,
            ),
        )
    
    # 임베딩 모델
    embeddings = ClovaXEmbeddings(model="bge-m3")
    
    # VectorStore 초기화
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings,
    )
    
    # Document 변환
    documents = []
    ids = []
    
    for chunk in chunks:
        doc = Document(
            page_content=chunk.content_resolved or chunk.content,
            metadata={
                # 식별
                "law_name": chunk.law_name,
                "law_id": chunk.law_id,
                "article_num": chunk.article_num,
                "article_title": chunk.article_title,
                
                # 원본 보존
                "content_original": chunk.content,
                
                # 계층 구조
                "paragraphs": chunk.paragraphs,
                
                # 참조 관계
                "internal_refs": chunk.internal_refs,
                "external_refs": chunk.external_refs,
                "parent_law_refs": chunk.parent_law_refs,
                
                # 축약어맵
                "abbreviations": chunk.abbreviations,
                
                # 메타
                "effective_date": chunk.effective_date,
                "change_type": chunk.change_type,
                "law_type": classify_law_type(chunk.law_name),
            },
        )
        documents.append(doc)
        ids.append(f"{chunk.law_name}_{chunk.article_num}")
    
    # 저장
    vector_store.add_documents(documents=documents, ids=ids)
    
    print(f"✅ {len(documents)}개 조문 저장 완료 → {collection_name}")
    
    return vector_store


def classify_law_type(law_name: str) -> str:
    if "시행규칙" in law_name:
        return "시행규칙"
    elif "시행령" in law_name:
        return "시행령"
    else:
        return "법률"

In [30]:
law_name = data["법령"]["기본정보"]["법령명_한글"]
print(f"📖 {law_name} 처리 시작...")

# STEP 1-2: 조문 파싱
chunks = parse_law_data(data)

📖 건축법 처리 시작...


TypeError: sequence item 3: expected str instance, list found

In [ ]:
def build_law_db(data: dict, collection_name: str = "building_law"):
    """
    법령 API 응답 → 파싱 → 전처리 → Qdrant 저장
    전체 파이프라인 실행
    """
    
    law_name = data["법령"]["기본정보"]["법령명_한글"]
    print(f"📖 {law_name} 처리 시작...")
    
    # STEP 1-2: 조문 파싱
    chunks = parse_law_data(data)
    print(f"   파싱 완료: {len(chunks)}개 조문")
    
    # STEP 1-3: 축약어맵 추출
    abbr_map = extract_abbreviations(chunks)
    print(f"   축약어 추출: {len(abbr_map)}개")
    for short, full in abbr_map.items():
        print(f"     '{short}' → '{full}'")
    
    # STEP 1-4: 참조 추출
    extract_references(chunks, law_name)
    total_refs = sum(
        len(c.internal_refs) + len(c.external_refs) + len(c.parent_law_refs) 
        for c in chunks
    )
    print(f"   참조 추출: 총 {total_refs}개")
    
    # STEP 1-5: 축약어 치환
    resolve_abbreviations(chunks, abbr_map)
    print(f"   축약어 치환 완료")
    
    # STEP 1-6: Qdrant 저장
    vector_store = store_to_qdrant(chunks, collection_name)
    print(f"✅ {law_name} 완료!\n")
    
    return vector_store, chunks


# 실행
# 건축법
vs1, chunks1 = build_law_db(building_law_data)

# 건축법 시행령 (별도 API 호출 후)
# vs2, chunks2 = build_law_db(building_decree_data)